<a href="https://colab.research.google.com/github/erickhiu/mario-npc/blob/main/omi_prelim_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [8]:
import pandas as pd
import plotly.express as px

# Data Preprocessing

In [9]:
from google.colab import drive
drive.mount('/content/drive')

product_df = pd.read_csv("/content/drive/MyDrive/2025 Job Search/OMI Assessment/Product Sales Report.csv")
order_df = pd.read_csv("/content/drive/MyDrive/2025 Job Search/OMI Assessment/Sales By Order.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# # Reading directly from uploaded files
# product_df = pd.read_csv("Product Sales Report.csv")
# order_df = pd.read_csv("Sales By Order.csv")

In [11]:
product_df['Category'] = product_df['Category'].fillna('Uncategorized')

# Question 1: POS Report Analysis


## 1A. Dashboard Design

### Metric 1: Total Sales (Overall)

- Gives management a quick snapshot of overall performance.
- If the dataset includes a date or timestamp column (e.g., transaction date), this metric can become more powerful by plotting a line chart of total sales over time. This helps to indentify trends (consistent growth or decline), spotting seasonal patterns, or even correlate with the impact of marketing campaigns.


In [12]:
total_sales = product_df['Total Sales'].sum()
print(f"The total sales is RM {total_sales}")

The total sales is RM 230014.38


### Metric 2: Top 10 Products by Sales (Standard vs Variant)
- This metric highlights that generate the most revenue.
- By separting standard vs variant sales using a stacked bar chart, the visualization could reveal upsell potential or certain variant of a product.
- This support pricing, bundling, and promotional decisions.

In [13]:
sales_compare = product_df.groupby('Product')[['Sales', 'Variant Sales']].sum()
sales_compare = sales_compare.rename(columns={
    'Sales': 'Standard Sales',
    'Variant Sales': 'Variant Sales'
})
sales_compare['Total Sales'] = sales_compare['Standard Sales'] + sales_compare['Variant Sales']
sales_compare = sales_compare.sort_values('Total Sales', ascending=False).head(10).reset_index()

fig = px.bar(
    sales_compare,
    y='Product',
    x=['Standard Sales', 'Variant Sales'],
    orientation='h',
    title='Top 10 Products: Standard vs Variant Sales',
    labels={'value': 'RM'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(hovertemplate='RM %{x:.2f} ')
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.show()

### Metric 3: Top 10 Products by Quantity (Standard vs Variant)
- This metric reveals operationally important items (e.g., preparation workload).
- This metric also reveals products that sell often but may be underpriced.
- Similar to Top 10 Products by Sales, the stacked bar chart also indicates customer preference behavior over the standard or variant version of the product.

In [14]:
quantity_compare = product_df.groupby('Product')[['Quantity', 'Variant Quantity']].sum()
quantity_compare = quantity_compare.rename(columns={
    'Quantity': 'Standard Quantity',
    'Variant Quantity': 'Variant Quantity'
})
quantity_compare['Total Quantity'] = (
    quantity_compare['Standard Quantity'] + quantity_compare['Variant Quantity']
)
quantity_compare = quantity_compare.sort_values('Total Quantity', ascending=False).head(10).reset_index()

fig = px.bar(
    quantity_compare,
    y='Product',
    x=['Standard Quantity', 'Variant Quantity'],
    orientation='h',
    title='Top 10 Products: Standard vs Variant Quantity',
    labels={'value': 'Units'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(hovertemplate='%{x:.0f} units')

fig.update_layout(yaxis=dict(autorange='reversed'))

fig.show()

### Metric 4: Sales by Category
- This metric helps management understand the contribution of each product category to overall sales.
- This will be useful for menu startegy decisions. For example, if a certain category contribute less than expected, they may need redesignining or repricing.
- A pie chart is used to visualize this metric because it communicates category share clearly at a glance.
- To improve readability and focus, small categories were grouped under a single "Others" segment in the pie chart.

In [15]:
cat_sales = product_df.groupby('Category')['Total Sales'].sum().reset_index()

cat_sales = cat_sales.sort_values('Total Sales', ascending=False)
cat_sales['Percent'] = cat_sales['Total Sales'] / cat_sales['Total Sales'].sum()

threshold = 0.015  # 1.5%, customize as needed
big_cats = cat_sales[cat_sales['Percent'] >= threshold].copy()
small_cats = cat_sales[cat_sales['Percent'] < threshold].copy()

others_row = pd.DataFrame({
    'Category': ['Others'],
    'Total Sales': [small_cats['Total Sales'].sum()],
    'Percent': [small_cats['Percent'].sum()]
})

cat_sales_clean = pd.concat([big_cats, others_row], ignore_index=True)

fig = px.pie(
    cat_sales_clean,
    names='Category',
    values='Total Sales',
    title='Sales Distribution by Category (Grouped Small Categories)',
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(
    textinfo='label+percent',
    hovertemplate='Category: %{label}<br>Sales: RM %{value:,.2f}<br>Percent: %{percent}'
)

fig.show()

## 1B. Product Contribution Analysis

In [16]:
wagyu_mask = (
    product_df['Product'].str.contains('Wagyu', case=False, na=False) |
    product_df['Category'].str.contains('Wagyu', case=False, na=False)
)

wagyu_df = product_df[wagyu_mask].copy()

wagyu_sales = wagyu_df['Total Sales'].sum()

total_sales = product_df['Total Sales'].sum()

wagyu_percent = wagyu_sales / total_sales * 100

print(f"💹 Wagyu-related items contributed RM {wagyu_sales:,.2f} to total sales.")
print(f"📊 This represents {wagyu_percent:.2f}% of all sales.")

💹 Wagyu-related items contributed RM 46,374.90 to total sales.
📊 This represents 20.16% of all sales.


## 1C. Sales Performance Metric

In my view, both sales amount and quantity sold are important, and which one to prioritize depends on the objective of the analysis and the product type.

For instance, if our goal is to analyze customer demand or product popularity **over time**, or when prices are volatile (due to discounts, inflation, exchaneg rate, seasonal menus, etc.), then quantity sold is a better metric. For example, the Sudoku Platter might have a lower sales due to a seasonal promotion, but consistent quantity sold still shows that it's a staple favorite. From operation's persepctive, quantity sold also provides insights for planning inventory and staffing. Looking solely at sales amount (which drop due to some external factor) may be misleading in informing operational decisions.

On the other hand, sales amount is more useful when comparing the financial performance between items. It helps track revenue contribution, identify high-margin products, and inform menu pricing or promotional strategies. For example, while the Sudoku Platter rank first by sales, it is not in teh top 10 products by quantity. Meanwhile Signature Teriyaki sells in high volumes but generate much less revenue. These insights could help buiness to identify items to upsell or bundle (e.g., high revenue Wagyu dishes), or to adjust pricing for some products.


In conclusion, rather than choosing one over the other, I would track both to understand what customers love while keeping the buisiness profitable.

## 1D. Product Development Recommendation

# Archive

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# 1. Aggregate and compute unit price
bucket_df = product_df.groupby(['Product', 'Category']).agg({
    'Total Quantity': 'sum',
    'Total Sales': 'sum'
}).reset_index()

bucket_df['Unit Price'] = bucket_df['Total Sales'] / bucket_df['Total Quantity']

# 2. Compute medians
x_median = bucket_df['Total Quantity'].median()
y_median = bucket_df['Unit Price'].median()

# 3. Classify into buckets
def classify(row):
    if row['Total Quantity'] >= x_median and row['Unit Price'] >= y_median:
        return "High Sales, High Margin"
    elif row['Total Quantity'] < x_median and row['Unit Price'] >= y_median:
        return "Low Sales, High Margin"
    elif row['Total Quantity'] >= x_median and row['Unit Price'] < y_median:
        return "High Sales, Low Margin"
    else:
        return "Low Sales, Low Margin"

bucket_df['Bucket'] = bucket_df.apply(classify, axis=1)

# 4. Prepare traces
def make_trace(df, name):
    return go.Scatter(
        x=df['Total Quantity'],
        y=df['Unit Price'],
        mode='markers',
        marker=dict(size=10),
        name=name,
        text=df['Product'],
        hovertemplate="<b>%{text}</b><br>Quantity: %{x}<br>Unit Price: %{y:.2f}<extra></extra>",
        showlegend=False
    )

# Base traces
traces = [
    make_trace(bucket_df, 'All Products'),
    make_trace(bucket_df.nlargest(10, 'Total Sales'), 'Top 10 by Sales'),
    make_trace(bucket_df.nlargest(20, 'Total Sales'), 'Top 20 by Sales')
]

# Per-category traces
categories = bucket_df['Category'].dropna().unique()
for cat in categories:
    df_cat = bucket_df[bucket_df['Category'] == cat]
    traces.append(make_trace(df_cat, f"Category: {cat}"))

# 5. Layout and dropdown menu
buttons = [
    dict(label="All Products", method="update", args=[{"visible": [True] + [False]* (len(traces)-1)}]),
    dict(label="Top 10 by Sales", method="update", args=[{"visible": [False, True] + [False]* (len(traces)-2)}]),
    dict(label="Top 20 by Sales", method="update", args=[{"visible": [False, False, True] + [False]* (len(traces)-3)}])
]

# Add buttons for each category
for i, cat in enumerate(categories):
    visible = [False]*len(traces)
    visible[3 + i] = True  # category traces start at index 3
    buttons.append(
        dict(label=f"Category: {cat}", method="update", args=[{"visible": visible}])
    )

# 6. Create figure
fig = go.Figure(data=traces)

# Add quadrant lines
fig.add_vline(x=x_median, line_dash="dash", line_color="gray")
fig.add_hline(y=y_median, line_dash="dash", line_color="gray")

# Layout updates
fig.update_layout(
    title="Profitability Buckets: Sales Volume vs Unit Price",
    xaxis_title="Total Quantity Sold",
    yaxis_title="Unit Price (RM)",
    height=650,
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        showactive=True,
        x=1.0,
        xanchor="right",
        y=1.15,
        yanchor="top"
    )]
)

fig.show()